In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import glob
import json
import pickle
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from io import BytesIO
import base64

[nltk_data] Downloading package punkt to /Users/dhv001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', -1)

In [3]:
# data.pkl from Kaggle notebook
df = pd.read_pickle('data.pkl')

In [4]:
len(df)

1612

In [5]:
df.head()

paper_id  \
0  14d04f36cb13550aa7769b61a079fa54031a21eb   
1  9937968a45f1947fa5d847f2942a0fe8a30ac280   
2  93e07ba40700d721247683851f5bc1c6c141fc97   
3  e2c96eee03031c4553a4412e86ed9655681c03f5   
4  c3a60d9d180f6533c76840edaf8b5216ecf357d8   

                                                                                                                                               title  \
0  risk management and health care policy                                                                                                              
1  impact of quarantine on the 2003 sars outbreak: a retrospective modeling study                                                                      
2  evaluation of multiple commercial molecular and conventional diagnostic assays for the detection of respiratory viruses in children                 
3  intractable & rare diseases research                                                                                                                
4  person to person droplets transmission characteristics in unidirectional ventilated protective isolation room: the impact of initial droplet size   

                                                                                                                            author_list  \
0  [Melanie Taylor, Beverley Raphael, Margo Barr, Kingsley Agho, Garry Stevens, Louisa Jorm]                                              
1  [Ying-Hen Hsieh, Chwan-Chuan King, Cathy W S Chen, Mei-Shang Ho, Sze-Bi Hsu, Yi-Chun Wu]                                               
2  [F Gharabaghi, A Hawan, S J Drews, S E Richardson]                                                                                     
3  [Sayantan Banerjee, Nitin Gupta, Parul Kodan, Ankit Mittal, Yogiraj Ray, Neeraj Nischal, Manish Soneja, Ashutosh Biswas, Naveet Wig]   
4  [Caiqing Yang, Xudong Yang, Bin Zhao]                                                                                                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 abstract  \
0  this research assessed factors associated with willingness to comply with vaccination, isolation, and face mask wearing during an anticipated infl uenza pandemic. data were collected from 2081 adults (16+) using a module of questions incorporated into the nsw health adult population health survey. high levels of willingness to comply were reported with 73% either very or extremely willing to receive vaccination, 67% willing to isolate themselves, 58% willing to we

In [6]:
class Preprocesser:
    def __init__(self, df):
        self.keywords = ['incident command system',
                         'emergency operations',
                         'joint information center',
                         'social distancing',
                         'childcare closers',
                         'travel advisory',
                         'travel warning',
                         'isolation',
                         'quarantine',
                         'mass gathering cancellations',
                         'school closures',
                         'facility closures',
                         'evacuation',
                         'relocation',
                         'restricting travel',
                         'travel ban',
                         'patient cohort',
                         'npi']
        self.occurances_minimum = 2
        self.df_full = df
        print(self.df_full.shape)
        self.key_slice()
        print(self.df_full.shape)
        self.npi_slice()
        print(self.df_full.shape)
    
    def key_slice(self):
        self.df_full = self.df_full[self.df_full['abstract'].str.contains('|'.join(self.keywords), na=False, regex=True)].reset_index(drop=True)
        
    def npi_slice(self):
        def get_count(row):
            return sum([row['abstract'].count(keyword) for keyword in self.keywords])
        self.df_full = self.df_full[self.df_full.apply(get_count, axis=1) >= self.occurances_minimum]
        
    def remove_stopwords(self,columns):
        stop = stopwords.words('english')
        for col in columns:
            self.df_full[col] = self.df_full[col].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    def to_tfidf(self,columns):
        for col in columns:
            tfidfv = TfidfVectorizer()
            self.df_full[col + '_tfidf'] = list(tfidfv.fit_transform(self.df_full[col]).toarray())
            
    def remove_punc(self, columns):
        for col in columns:
            self.df_full[col] = self.df_full[col].str.replace('[^a-zA-Z\s]+','')

In [7]:
def pca_apply(df, columns, n_comp):
    new_df = df.copy()
    for col in columns:
        pca = PCA(n_components=n_comp, random_state=1)
        new_df[col+'_pca'] = list(pca.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def apply_scaler(df, columns):
    new_df = df.copy()
    for col in columns:
        scaler = StandardScaler()
        new_df[col + '_scaled'] = list(scaler.fit_transform(np.stack(df[col].to_numpy())))
    return new_df.reset_index(drop=True)

def tokenize(row):
    title_tokens = []
    title = row['abstract']
    if title == title:
        title = re.sub('(/|\|:|&|#|-|\.)', '', title)
        tokens = word_tokenize(title)
        remove_sw = [word for word in tokens if word not in stopwords.words('english')]
        remove_numbers = [word for word in remove_sw if not word.isnumeric()]
        remove_comas = [word for word in remove_numbers if not word in [',', '(', ')', '"', ':', '``', '.', '?']]
        title_tokens.extend(remove_comas)
    return [value[0] for value in Counter(title_tokens).most_common()[0:30]]

In [8]:
prepr = Preprocesser(df)

(1612, 6)
(1612, 6)
(520, 6)


In [9]:
prepr.remove_punc(['body_text','abstract'])
prepr.remove_stopwords(['body_text', 'abstract'])
prepr.to_tfidf(['body_text', 'abstract'])
pca_df = pca_apply(prepr.df_full, ['abstract_tfidf','body_text_tfidf'], 3)
scaled_df = apply_scaler(pca_df,['abstract_tfidf_pca','body_text_tfidf_pca'])
scaled_df['tokens'] = scaled_df.apply(tokenize, axis=1)

In [11]:
# from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

def get_breaks(row, col, word_limit=45, break_char='<br>', colon=True):
    if row[col] == row[col]:
        data = ''
        if col in ['tokens', 'author_list', 'doi']:
            if col != 'tokens':
                data = f'**{col.capitalize()}**: '
            words = row[col]
        elif colon:
            data = f'**{col.capitalize()}**: '
            words = row[col].split(' ')
        else:
            words = row[col].replace('<br>', '').split(' ')
        total_chars = 0

        # add break every length characters
        for i in range(len(words)):
            total_chars += len(words[i])
            if total_chars > word_limit:
                data += f'{break_char}{words[i]}'
                total_chars = 0
            else:
                if col in ['tokens', 'author_list', 'doi'] and data:
                    data += f' {words[i]},'
                else:
                    data += f' {words[i]}'
        return data.strip().rstrip(',')
    return row[col]

def display_wordcloud(text):
    wc= WordCloud(max_font_size=50,
                          max_words=100,
                          background_color='white',
                          width=480, height=360).generate(text)
    return wc.to_image()

class Cluster_Plot:
    def __init__(self, df, text_type, clust_nums):
        self.styles = {
            'pre': {
                'border': 'thin lightgrey solid',
                'overflowX': 'scroll'
            }
        }
        self.set_app()
        self.df = df
        self.set_text(text_type)
        self.clust_nums = clust_nums
        self.create_cluster_df()
        
        if self.app is not None and hasattr(self, 'callbacks'):
            self.callbacks(self.app)

    def run_process(self):
        self.set_app_layout()
        self.app.run_server()
        
    def set_app(self):
        self.app = dash.Dash(__name__,
                        external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css", "../localstyles.css"])
        
    def set_app_layout(self):
        self.app.layout = html.Div(children=[
            html.Div(className='row', style={'background-color': '#142a57'}, children=[
                html.Div(className='container', style={'max-width': 'unset'}, children=[
                    html.Div(className='row', children=[
                        html.Div(className='nine columns', children=[
                            html.H1('NPI Cluster Analysis', style={ 'color': 'white', 'padding-top': '1%'})
                        ]),
                        html.Div(className='three columns', children=[
                            html.Img(
                                src='https://storage.googleapis.com/lab_assets/Jefferson_CorporateEnterpriseLightBlue-01.png',
                                style={ 'width': '60%', 'padding-top': '5%'}
                            )
                        ])
                    ])
                ])
            ]),
            html.Div(className='container', style={'max-width': 'unset'}, children=[
                html.Div(className='row', children=[
                    html.Div(className='three columns', children=[
                        html.H2('Cluster Control Panel'),
                        html.Label('Select to cluster on the article abstract or body'),
                        dcc.RadioItems(
                            id='abstract_or_body',
                            options=[{'label': 'abstract', 'value': 'abstract'},
                                     {'label': 'body', 'value': 'body_text'}],
                            value=self.text_type
                        ),
                        html.Label('Select number of Clusters'),
                        dcc.Dropdown(
                            id='cluster_num',
                            options=[{'label': i+1, 'value': i+1} for i in list(range(20))],
                            value=self.clust_nums
                        ),
                        html.Label('Select cluster to view'),
                        dcc.Dropdown(
                            id='cluster_id',
                            options=self.cluster_id_list,
                            value='all'
                        ),
                    ]),
                    html.Div(className='six columns', children=[
                        dcc.Graph(id="graph", style={"width": "90%", "display": "inline-block"})
                    ]),
                    html.Div(className='six columns', children=[
                        html.H3('Word Cloud for clicked data point'),
                        html.Img(id="image_wc")]),
                    html.Div(className='three columns', children=[
                        html.H2('Filter Articles'),
                        html.Label('Search keywords'),
                        dcc.Input(id='search',
                            value='',
                            type='text'
                        )
                    ]),
                ]),
                html.Div(className='row', children=[
                    html.Div([
                        dcc.Markdown("""
                            **Selected Article**

                            Click on values in the plot to select article.
                        """),
                        dcc.Markdown(id='hover-data', style=self.styles['pre'])
                    ])
                ])
            ]),
            html.Div(className='row', style={'background-color': '#142a57', 'position': 'fixed', 'bottom': '0'}, children=[
                html.Div(style={'max-width': 'unset'}, children=[
                    html.Img(
                                src='https://storage.googleapis.com/lab_assets/DICE_PoweredByEnhancedWhite.png',
                                style={ 'width': '10%', 'padding-top': '1%', 'margin-left': '45%', 'margin-right': '45%'}
                            )
                ])
            ])
        ])
        
    def create_cluster_df(self):
        self.cluster_id_list = [{'label': i, 'value': i} for i in list(range(self.clust_nums))]
        self.cluster_id_list.append({'label': 'all', 'value': 'all'})
        new_df = self.df.copy()
        kmeans = KMeans(n_clusters = self.clust_nums, random_state=1)
        new_df[self.col_cluster_id] = list(kmeans.fit_predict(np.stack(new_df[self.col].to_numpy())))
        self.cluster_df = new_df.reset_index(drop=True)
        self.cluster_df['title'] = self.cluster_df.apply(get_breaks, args=('title',), axis=1)
        self.cluster_df[['x', 'y', 'z']] = pd.DataFrame(self.cluster_df[self.col].values.tolist(),
                                                        index = self.cluster_df.index)
    
    def set_text(self, text_type):
        self.text_type = text_type
        self.col = f'{self.text_type}_tfidf_pca_scaled'
        self.col_cluster_id = f'{self.text_type}_tfidf_pca_scaled_clusterID'
        
    def callbacks(self, app):
        @app.callback([Output('cluster_id', 'options'),
                       Output('graph', 'figure')],
                      [Input('abstract_or_body', 'value'),
                       Input('cluster_num', 'value'),
                       Input('cluster_id', 'value'),
                       Input('search', 'value')])
        def update_clusters(abstract_or_body, cluster_num, cluster_id, search_string):
            self.set_text(abstract_or_body)
            self.clust_nums = cluster_num
            self.create_cluster_df()
            options = self.cluster_id_list
            df = self.cluster_df.copy()
            show_scale = True
            if cluster_id != 'all':
                show_scale = False
                df = df[df[self.col_cluster_id] == cluster_id]
            df = df[df[self.text_type].str.contains(search_string, na=False, regex=True)].reset_index(drop=True)
            fig = px.scatter_3d(df, x='x', y='y', z='z',
                                 color=self.col_cluster_id,
                                 hover_name='title',
                                 hover_data=['paper_id', 'doi'])
            fig.update_layout(title = '3D cluster of research papers',
                              scene = dict(
                                xaxis = dict(nticks=4, range=[-5,5],),
                                yaxis = dict(nticks=4, range=[-5,5],),
                                zaxis = dict(nticks=4, range=[-5,5],),),
                              hoverlabel=dict(
                                bgcolor='white', 
                                font_size=8, 
                                font_family='Rockwell'
                              ),
                              coloraxis=dict(
                                colorbar=dict(title='Cluster ID'),
                                showscale=show_scale
                              ))
            return options, fig
        
        @app.callback([Output("hover-data", "children"),
                      Output('image_wc', 'src')],
                      [Input("graph", "clickData"),
                       Input('image_wc', 'id')])
        def display_click_data(clickData, b):
            string = None
            if clickData:
                click_index = clickData['points'][0]['pointNumber']
                token_string = ''
                if self.cluster_df.iloc[click_index]["tokens"]:
                    token_string = get_breaks(self.cluster_df.iloc[click_index],
                                              'tokens',
                                              word_limit=100,
                                              break_char='\n')
                    token_string = f'**Keywords**: {token_string}'
                    
                string = get_breaks(self.cluster_df.iloc[click_index],
                                    'title',
                                    word_limit=100,
                                    break_char='\n',
                                    colon=False)
                item_list = ['abstract', 'body_text', 'author_list', 'paper_id', 'doi']
                for i in item_list:
                    formatted_data = get_breaks(self.cluster_df.iloc[click_index],
                                               i,
                                               word_limit=100,
                                               break_char='\n')
                    string += f'\n\n{formatted_data}'
                img = BytesIO()
                display_wordcloud(self.cluster_df.iloc[click_index][self.text_type]).save(img, format='PNG')
                image = 'data:image/png;base64,{}'.format(base64.b64encode(img.getvalue()).decode())
                if token_string:
                    string = f'{token_string}\n\n{string}'
                return string, image
            return string, None

c = Cluster_Plot(scaled_df, 'abstract', 10)
c.run_process()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Apr/2020 17:27:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:39] "GET /localstyles.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2020 17:27:42] "POST /_dash-update-component HTTP/1.1" 200 -
